<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#ML-Pipeline-Preparation" data-toc-modified-id="ML-Pipeline-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>ML Pipeline Preparation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-Import-libraries-and-load-data-from-database." data-toc-modified-id="1.-Import-libraries-and-load-data-from-database.-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>1. Import libraries and load data from database.</a></span></li><li><span><a href="#2.-Write-a-tokenization-function-to-process-your-text-data" data-toc-modified-id="2.-Write-a-tokenization-function-to-process-your-text-data-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>2. Write a tokenization function to process your text data</a></span></li><li><span><a href="#3.-Build-a-machine-learning-pipeline" data-toc-modified-id="3.-Build-a-machine-learning-pipeline-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>3. Build a machine learning pipeline</a></span></li><li><span><a href="#4.-Train-pipeline" data-toc-modified-id="4.-Train-pipeline-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>4. Train pipeline</a></span></li><li><span><a href="#5.-Test-your-model" data-toc-modified-id="5.-Test-your-model-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>5. Test your model</a></span></li><li><span><a href="#6.-Improve-your-model" data-toc-modified-id="6.-Improve-your-model-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>6. Improve your model</a></span></li><li><span><a href="#7.-Test-your-model" data-toc-modified-id="7.-Test-your-model-1.0.7"><span class="toc-item-num">1.0.7&nbsp;&nbsp;</span>7. Test your model</a></span></li><li><span><a href="#8.-Try-improving-your-model-further.-Here-are-a-few-ideas:" data-toc-modified-id="8.-Try-improving-your-model-further.-Here-are-a-few-ideas:-1.0.8"><span class="toc-item-num">1.0.8&nbsp;&nbsp;</span>8. Try improving your model further. Here are a few ideas:</a></span></li><li><span><a href="#9.-Export-your-model-as-a-pickle-file" data-toc-modified-id="9.-Export-your-model-as-a-pickle-file-1.0.9"><span class="toc-item-num">1.0.9&nbsp;&nbsp;</span>9. Export your model as a pickle file</a></span></li><li><span><a href="#10.-Use-this-notebook-to-complete-train.py" data-toc-modified-id="10.-Use-this-notebook-to-complete-train.py-1.0.10"><span class="toc-item-num">1.0.10&nbsp;&nbsp;</span>10. Use this notebook to complete <code>train.py</code></a></span></li></ul></li></ul></li></ul></div>

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [283]:
# import libraries
import re
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import itertools

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import gensim
import gensim.downloader
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wahyu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [130]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table(con=engine,table_name="disaster_tweet")
X = df['message']
Y = df.iloc[:,3:].values

In [131]:
list_col = [col.split('_') for col in df.iloc[:,3:].columns]
synonym_to_find = set(list(itertools.chain(*list_col)))

### 2. Write a tokenization function to process your text data

In [226]:
def tokenize(text):
    # Remove non alphanumeric char
#     text = re.sub(r'[^a-zA-Z0-9]', '', text)

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    clean_tokens = [w for w in clean_tokens if not w.lower() in stop_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [306]:
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier',MultiOutputClassifier(RandomForestClassifier(n_estimators=2,min_samples_split=30))),    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [307]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.75, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [308]:
output_category = list(df.iloc[:,3:].columns)
Y_pred_train = pipeline.predict(X_train)
for i in range(Y_train.shape[1]):
    print(output_category[i])
    print(classification_report(Y_train.T[i], Y_pred_train.T[i]))

related
              precision    recall  f1-score   support

           0       0.79      0.78      0.78      1513
           1       0.93      0.94      0.94      5040

    accuracy                           0.90      6553
   macro avg       0.86      0.86      0.86      6553
weighted avg       0.90      0.90      0.90      6553

request
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      5452
           1       0.87      0.67      0.76      1101

    accuracy                           0.93      6553
   macro avg       0.90      0.83      0.86      6553
weighted avg       0.93      0.93      0.92      6553

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.88      0.27      0.41        26

    accuracy                           1.00      6553
   macro avg       0.94      0.63      0.71      6553
weighted avg       1.00      1.00      1.00      655

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5985
           1       0.90      0.63      0.74       568

    accuracy                           0.96      6553
   macro avg       0.93      0.81      0.86      6553
weighted avg       0.96      0.96      0.96      6553

fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6478
           1       0.96      0.33      0.50        75

    accuracy                           0.99      6553
   macro avg       0.98      0.67      0.75      6553
weighted avg       0.99      0.99      0.99      6553

earthquake
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      5982
           1       0.92      0.60      0.73       571

    accuracy                           0.96      6553
   macro avg       0.94      0.80      0.85      6553
weighted avg       0.96      0.96      0.96      6553

col

In [309]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
              precision    recall  f1-score   support

           0       0.42      0.38      0.40      4608
           1       0.82      0.84      0.83     15053

    accuracy                           0.73     19661
   macro avg       0.62      0.61      0.61     19661
weighted avg       0.72      0.73      0.73     19661

request
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     16288
           1       0.59      0.40      0.48      3373

    accuracy                           0.85     19661
   macro avg       0.73      0.67      0.69     19661
weighted avg       0.83      0.85      0.84     19661

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19569
           1       0.00      0.00      0.00        92

    accuracy                           0.99     19661
   macro avg       0.50      0.50      0.50     19661
weighted avg       0.99      0.99      0.99     1966

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     17777
           1       0.73      0.33      0.45      1884

    accuracy                           0.92     19661
   macro avg       0.83      0.66      0.71     19661
weighted avg       0.91      0.92      0.91     19661

cold
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     19255
           1       0.23      0.03      0.05       406

    accuracy                           0.98     19661
   macro avg       0.61      0.51      0.52     19661
weighted avg       0.96      0.98      0.97     19661

other_weather
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     18620
           1       0.17      0.04      0.07      1041

    accuracy                           0.94     19661
   macro avg       0.56      0.52      0.52     19661
weighted avg       0.91      0.94      0.92     19661



### 6. Improve your model
Use grid search to find better parameters. 

In [310]:
parameters = {
    'count_vectorizer__ngram_range': ((1, 1), (1, 2)),
    'classifier__estimator__n_estimators': [2,5,10,20],
    'classifier__estimator__min_samples_split': [2,5,10,20,30]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train,Y_train)

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [90]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [272]:
class SentenceVectorizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.transformer = SentenceTransformer('all-MiniLM-L6-v2')
        return self

    def transform(self, X):
        embeddings = self.transformer.encode(list(X))
        return embeddings

# class SynonymExtractor(BaseEstimator, TransformerMixin):
#     def __init__(self,synonyms,tokenizer,num_of_synonym=10):
#         self.num_of_synonym = num_of_synonym
#         self.model = gensim.downloader.load('glove-wiki-gigaword-300')
#         self.synonym_to_find = synonyms
#         self.tokenizer=tokenizer
    
#     def count_synonym(self,X):
#         count_result = []
#         X = pd.Series(X).apply(lambda row: self.tokenizer(row), axis=1)
#         for word in synonym_to_find:
#             synonys_list = [word for word, _ in glove_vectors.most_similar(positive=[word], topn = self.num_of_synonym)]
#             synonys_list = np.array(sorted(synonys_list))
#             sorted_text_token = np.array(sorted(text))

#             idx = np.searchsorted(synonys_list,sorted_text_token)
#             idx[idx==len(synonys_list)] = 0
#             mask = synonys_list[idx]==sorted_text_token
#             out = np.sum(np.bincount(idx[mask]))            
#             count_synonym = len(list(set(synonys_list) & set(text)))
#             count_result.append(count_synonym)
#         print(pd.Series(count_result))
#         return pd.Series(count_result)
#         pass
    
#     def fit(self, x, y=None):
#         return self
#     def transform(self, X, ):
#         X_tagged = pd.Series(X).apply(self.count_synonym)
#         return pd.DataFrame(X_tagged)

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
#         ('extractor', SynonymExtractor(synonym_to_find,tokenize)),
        ('sent2vec', SentenceVectorizer()),
    ])),

    ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=3))),
])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.75, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

In [273]:
output_category = list(df.iloc[:,3:].columns)
Y_pred_train = pipeline.predict(X_train)
for i in range(Y_train.shape[1]):
    print(output_category[i])
    print(classification_report(Y_train.T[i], Y_pred_train.T[i]))

related
              precision    recall  f1-score   support

           0       0.93      0.84      0.88      1513
           1       0.95      0.98      0.97      5040

    accuracy                           0.95      6553
   macro avg       0.94      0.91      0.93      6553
weighted avg       0.95      0.95      0.95      6553

request
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5452
           1       0.95      0.85      0.90      1101

    accuracy                           0.97      6553
   macro avg       0.96      0.92      0.94      6553
weighted avg       0.97      0.97      0.97      6553

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       1.00      0.73      0.84        26

    accuracy                           1.00      6553
   macro avg       1.00      0.87      0.92      6553
weighted avg       1.00      1.00      1.00      655

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4813
           1       0.95      0.89      0.92      1740

    accuracy                           0.96      6553
   macro avg       0.95      0.94      0.95      6553
weighted avg       0.96      0.96      0.96      6553

floods
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6036
           1       0.97      0.83      0.89       517

    accuracy                           0.98      6553
   macro avg       0.98      0.91      0.94      6553
weighted avg       0.98      0.98      0.98      6553

storm
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5985
           1       0.96      0.83      0.89       568

    accuracy                           0.98      6553
   macro avg       0.97      0.91      0.94      6553
weighted avg       0.98      0.98      0.98      6553

fire
 

In [274]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
              precision    recall  f1-score   support

           0       0.44      0.28      0.35      4608
           1       0.80      0.89      0.84     15053

    accuracy                           0.75     19661
   macro avg       0.62      0.59      0.59     19661
weighted avg       0.72      0.75      0.73     19661

request
              precision    recall  f1-score   support

           0       0.87      0.94      0.90     16288
           1       0.52      0.30      0.38      3373

    accuracy                           0.83     19661
   macro avg       0.69      0.62      0.64     19661
weighted avg       0.81      0.83      0.81     19661

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19569
           1       0.00      0.00      0.00        92

    accuracy                           1.00     19661
   macro avg       0.50      0.50      0.50     19661
weighted avg       0.99      1.00      0.99     1966

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     18620
           1       0.20      0.03      0.05      1041

    accuracy                           0.94     19661
   macro avg       0.58      0.51      0.51     19661
weighted avg       0.91      0.94      0.92     19661

direct_report
              precision    recall  f1-score   support

           0       0.84      0.92      0.88     15857
           1       0.46      0.28      0.35      3804

    accuracy                           0.80     19661
   macro avg       0.65      0.60      0.61     19661
weighted avg       0.77      0.80      0.78     19661



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.